# <b> NOTE BOOK PROJET DE CLASSIFICATION AUTOMATIQUE DES ENTRPRISES SELON LE CODE NAEMAS </b>

In [1]:

import os
os.chdir('C:/Users/mombengue/Desktop/REP_PYTHON')
os.getcwd()


'C:\\Users\\mombengue\\Desktop\\REP_PYTHON'

# <b> I. PREPARATION DES DONNEES </b>

In [2]:
# Installation du module pandas dans cet environnement
#! pip install pandas

In [3]:
# installation de la  dépendance openpyxl.
#!pip install openpyxl
# installation de nltk
#!pip install nltk

In [4]:
# Chargement de la base
import pandas as pd
base=pd.read_excel("repertoire_entreprise_2022.xlsx", sheet_name=0)
base.shape


(25643, 14)

In [5]:
# Selection des variables en jeu
df=base[["DESIGNATION_PRECISE_ACTIVITE","NAEMAS"]]
df.head(10)

,DESIGNATION_PRECISE_ACTIVITE,NAEMAS
0,BOULANGERIE,F00060
1,COMMERCE GENERAL,N00030
2,VENTE DE CARTES POSTALES (EX-EDITION),Q00050
3,SERVICE,N00020
4,OPTICIEN,J00140
5,LOCATION DE BIENS IMMOBILIERS,S00000
6,GARDIENNAGE,U00030
7,BTP,M00020
8,NaN,K00010
9,ACTIVITES POUR LA SANTE DES HOMMES (OPHTALMOLO...,X00010


In [6]:
df.shape

(25643, 2)

In [7]:
# Détection des lignes manquantes 
df.isnull().sum() 

DESIGNATION_PRECISE_ACTIVITE    856
NAEMAS                            5
dtype: int64

In [8]:
# Suppression des lignes manquantes
df= df.dropna()
df.isnull().sum()

DESIGNATION_PRECISE_ACTIVITE    0
NAEMAS                          0
dtype: int64

# <b> II. METHODE DE COMPARAISON DES VECTEURS DE MOTS </b>

# Principe

- D'abord, on regroupe sous forme de vecteurs de mots les descriptions des activités principales des entreprises ayant le même code NAEMAS. Ces vecteurs de mots sont donc étiquetés avec un code NAEMAS.

- Ensuite, pour chaque nouvelle entreprise à classer, on regroupe la description de son activité sous forme d'un vecteur de mots et on le compare aux différents vecteurs étiquetés.

- Enfin, l'entreprise à classer sera affectée au code NAEMAS du vecteur étiqueté dont elle est la plus proche (c'est à dire similaire).

# Metrique de comparaison
La comparaison des vecteurs se fait sur la base d'une métrique. La métrique utilisée pour comparer les vecteur est la <b> similarité cosinus </b>

La similarité cosinus mesure l'angle entre deux vecteurs dans un espace vectoriel. Elle est définie comme le produit scalaire des vecteurs divisé par le produit des normes des vecteurs. Elle varie entre -1 et 1 :

1 signifie que les vecteurs sont identiques (donc plus la mesure est proche de 1 plus le deux vecteurs sont similaires) <br>
0 signifie qu'ils sont orthogonaux (aucune similarité). <br>
-1 signifie qu'ils sont diamétralement opposés (inversion totale, ce qui est rare dans le contexte de similarité textuelle).


In [9]:
import nltk
from nltk.corpus import stopwords

# Télécharger les stop words de NLTK (à faire une seule fois)
nltk.download('stopwords')

# Récupérer la liste des stop words en français
stop_words = set(stopwords.words('french'))

# Fonction pour filtrer les stop words dans une chaîne de caractères

def remove_stopwords(text, stop_words):
    
    words = text.split()  # Tokenisation
    filtered_words = [word for word in words if word.lower() not in stop_words]
    return ' '.join(filtered_words)  # Reconstituer la chaîne de caractères

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\mombengue\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [10]:

# Récupération du code 'NAEMAS' sous forme de liste
Code = list(df['NAEMAS'].unique())

# Initialiser un dictionnaire vide pour stocker les résultats
Dicto = {}

# Parcourir les valeurs dans 'Code'
for k in Code:
    # Regroupement des descriptions d'activités ayant le même code NAEMAS
    desc = df[df['NAEMAS'] == k]['DESIGNATION_PRECISE_ACTIVITE'].tolist()

    # Appliquer la fonction remove_stopwords() à chaque chaîne de caractères pour chaque liste du dictionnaire

    filtered_list_of_strings = [remove_stopwords(sentence, stop_words) for sentence in desc]
    
    # Associer la liste filtered_list_of_strings au clé correspondant dans le dictionnaire
    Dicto[k] = filtered_list_of_strings 

# Afficher le résultat
#print(Dicto)


In [11]:
# installation de matplotlib pour cet environnement
#!pip install matplotlib

In [12]:
# installation de wordcloud dans l'environnement
#!pip install wordcloud

In [16]:

import matplotlib.pyplot as plt
from wordcloud import WordCloud

# Parcourir chaque clé et générer un nuage de mots pour chaque groupe

for key, activities in Dicto.items():
    
    # Combiner les activités en une seule chaîne de texte
    text = ' '.join(activities)
    
    # Créer un objet WordCloud
    
    wordcloud = WordCloud(width=800, height=400, background_color='white').generate(text)
    
    # Afficher le nuage de mots
    
   #plt.figure(figsize=(10, 5))
   # plt.imshow(wordcloud, interpolation='bilinear')
    #plt.title(f'Nuage de mots pour la clé {key}', pad=30)
   # plt.axis('off')
   # plt.show()


In [15]:
vecteurs = []
for k, activities in Dicto.items():
    
    # Combiner les activités en une seule chaîne de texte
     vect_k= ' '.join(activities)
     vecteurs.append(vect_k)



import numpy as np
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

vecteur_a_comparer = "PRESTATIONS DE SERVICES ( 63%)- VENTE DE MARCHANDISES (37%) -PRODUITS DIVERS"
# Utiliser le TfidfVectorizer
vectorizer = TfidfVectorizer()

# Convertir les vecteurs en une matrice TF-IDF
vecteurs_matrice = vectorizer.fit_transform(vecteurs + [vecteur_a_comparer])

# Calculer la similarité cosinus
similarites = cosine_similarity(vecteurs_matrice)

# Extraire les similarités du vecteur à comparer
similarites_vecteur_a_comparer = similarites[-1][:-1]

# Afficher les résultats
for i, similarite in enumerate(similarites_vecteur_a_comparer):
      print(f"Similarité entre vecteur4 et vecteur_{i+1}: {similarite:.4f}")

Similarité entre vecteur4 et vecteur_1: 0.0072
Similarité entre vecteur4 et vecteur_2: 0.1021
Similarité entre vecteur4 et vecteur_3: 0.0520
Similarité entre vecteur4 et vecteur_4: 0.0629
Similarité entre vecteur4 et vecteur_5: 0.0262
Similarité entre vecteur4 et vecteur_6: 0.0096
Similarité entre vecteur4 et vecteur_7: 0.0282
Similarité entre vecteur4 et vecteur_8: 0.0057
Similarité entre vecteur4 et vecteur_9: 0.0033
Similarité entre vecteur4 et vecteur_10: 0.1244
Similarité entre vecteur4 et vecteur_11: 0.0694
Similarité entre vecteur4 et vecteur_12: 0.0019
Similarité entre vecteur4 et vecteur_13: 0.0191
Similarité entre vecteur4 et vecteur_14: 0.0123
Similarité entre vecteur4 et vecteur_15: 0.0092
Similarité entre vecteur4 et vecteur_16: 0.0156
Similarité entre vecteur4 et vecteur_17: 0.0199
Similarité entre vecteur4 et vecteur_18: 0.0051
Similarité entre vecteur4 et vecteur_19: 0.0084
Similarité entre vecteur4 et vecteur_20: 0.0393
Similarité entre vecteur4 et vecteur_21: 0.0297
S

In [11]:
#! pip install scikit-learn

Explication du Code
CountVectorizer : Il convertit les textes en une matrice de fréquence de mots. Chaque mot dans le texte devient une caractéristique dans la matrice.

cosine_similarity : Cette fonction calcule la similarité cosinus entre les vecteurs de caractéristiques. La similarité cosinus est une mesure de similarité qui calcule le cosinus de l'angle entre deux vecteurs dans un espace multidimensionnel. C'est une mesure couramment utilisée pour comparer des documents textuels.

Matrice de Similarité : cosine_similarity retourne une matrice où chaque entrée [i, j] représente la similarité entre le i-ème et le j-ème vecteur.

Extraction des Similarités : La dernière ligne de la matrice contient les similarités entre le vecteur à comparer (vecteur_4) et tous les autres vecteurs.

Utilisation Avancée
Pour des analyses plus sophistiquées, vous pouvez également envisager d'autres techniques de vectorisation et de mesures de similarité, telles que :

Word2Vec : Pour obtenir des vecteurs de mots basés sur des modèles de langage plus complexes.
Doc2Vec : Pour représenter des documents (ou des nuages de mots) en tant que vecteurs.
BERT : Pour des représentations contextuelles des mots et des phrases.
Ces techniques peuvent capturer des nuances plus subtiles dans les textes et améliorer la précision de vos comparaisons. Voici un exemple utilisant TfidfVectorizer :

Le TfidfVectorizer est une classe de la bibliothèque scikit-learn en Python qui permet de convertir une collection de documents de texte brut en une matrice de caractéristiques TF-IDF (Term Frequency-Inverse Document Frequency).
Le TF-IDF est une technique utilisée pour évaluer l'importance d'un mot dans un document par rapport à un corpus de documents.

Comment Fonctionne le TF-IDF

Term Frequency (TF) : Mesure la fréquence d'un terme dans un document. Elle est souvent normalisée pour éviter un biais en faveur de longs documents.
​
Inverse Document Frequency (IDF) : Mesure l'importance d'un terme dans le corpus entier. Elle diminue le poids des termes très fréquents dans le corpus et augmente celui des termes rares.

TF-IDF : Combinaison des deux mesures, calculée comme le produit de TF et IDF :

TF-IDF(t,d,D)=TF(t,d)×IDF(t,D)

Utilisation de TfidfVectorizer en Python
Voici un exemple de code complet utilisant TfidfVectorizer pour transformer des textes en vecteurs TF-IDF et comparer leur similarité :